## MTH9878 Interest Rate Models HW1
* Group X
* Author: Pan, Hongchao & Zhang, Chendi
* Kernel version: Python 2.7
* Packages: 
* Data: 
* Notes:

In [2]:
# load packages and features from version 3.x
from __future__ import absolute_import, print_function, division
import pandas as pd
import numpy as np
import math

### Answer of Q1

In [4]:
# Define a function to calculate the faction of the date differnence
def date_fraction(t1,t2):
    # Assume input t1 and t2 are calendar dates
    days=(t2-t1).days
    # Use the date convention: actual/360
    return days/360  

In [ ]:
# Define a funtion to compute the integration by using simposon's rule
# a: left endpoint; b: right endpoint; n: number of partition intervals
# f: instantaneous rate function, f(s)
def simpson_rule(a,b,f,n):
    h= (b-a)/n
    result=(f(a) + f(b))/6

    for i in xrange(1, n):
        result += f(a + i*h)/3
    for i in xrange(1, n+1):
        result += 2*f(a + (i-0.5)*h)/3

    return result*h

In [5]:
# Define a function to compute the discount factor
def dist(t,T,f,flag):
    # t: starting date
    # T: maturity
    # f: interest rate
    # flag: indicator of f is constant rate or instantaneous rate
    
    # Assume input t and T are calendar dates
    
    # Get the day fraction
    day_frac=date_fraction(t,T)
    if(flag=='c'):
        # constant interest rate
        return math.exp(-f*day_frac)
    if(flag=='i'):
        # instantaneous rate
        print("Please define the instantaneous rate first.")
        return -1
    else:
        print("The input of interest rate is not well defined.")
        return -2

### Answer of Q2